In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import numpy as np

2022-02-08 16:47:45.803324: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-08 16:47:45.803385: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class MyDense(layers.Layer):

    def __init__(self, units=32, activation=None):
        super(MyDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        b_init = tf.zeros_initializer()

        # print(input_shape)

        self.w = tf.Variable(name='kernel',
                            initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'),
                            trainable=True)
        self.b = tf.Variable(name='bias',
                            initial_value=b_init(shape=(self.units, ), dtype='float32'),
                            trainable=True)

    def call(self, inputs):
        # print(inputs.shape, self.w.shape)
        return self.activation(tf.matmul(inputs, self.w) + self.b)

   
# my_dense = MyDense(units=1)
# x = tf.ones((1, 1))
# y = my_dense(x)
# print(my_dense.variables)

In [3]:
class Quad(layers.Layer):
    
    def __init__(self, units, activation=None):
        super(Quad, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        a_init = tf.random_normal_initializer()
        b_init = tf.random_normal_initializer()
        c_init = tf.zeros_initializer()

        self.a = tf.Variable(name='a',
                            initial_value = a_init(shape=(input_shape[-1], self.units), dtype='float32'),
                            trainable=True)

        self.b = tf.Variable(name='b',
                            initial_value = b_init(shape=(input_shape[-1], self.units), dtype='float32'),
                            trainable=True)

        self.c = tf.Variable(name='c',
                            initial_value = c_init(shape=(self.units, ), dtype='float32'),
                            trainable=True)

    def call(self, inputs):
        return tf.matmul(tf.math.square(inputs), self.a) + tf.matmul(inputs, self.b) + self.c


# my_dense = Quad(units=1)
# x = tf.ones((1, 1))
# y = my_dense(x)
# print(my_dense.variables)

In [13]:
tf.random_normal_initializer()

In [5]:
x = np.arange(10).astype(np.float32)[..., np.newaxis]
y = x*2-3

# print(x.shape)

my_dense = MyDense(units=1, activation='relu')
model = tf.keras.models.Sequential([my_dense])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(x, y, epochs=500, verbose=0)
print(model.predict([10.0]))
print(my_dense.variables)

[[16.618803]]
[<tf.Variable 'my_dense_1/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.8932732]], dtype=float32)>, <tf.Variable 'my_dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([-2.313929], dtype=float32)>]


In [7]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, X_test = X_train/255., X_test/255.

model = Sequential([
    layers.Flatten(input_shape=(28, 28)),
    Quad(units=128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)
model.evaluate(X_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3927 - accuracy: 0.8878
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3278 - accuracy: 0.9087
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3123 - accuracy: 0.9132
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3035 - accuracy: 0.9164
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.2862 - accuracy: 0.9205


[0.2862119674682617, 0.9204999804496765]